## DSA 2 Class Performance 2 - Solutions

### Problem 1: Maximum Value Subset (0/1 Knapsack Variant)

In [ ]:
def knapsack():
    n, W = map(int, input().split())
    items = []
    for _ in range(n):
        w, v = map(int, input().split())
        items.append((w, v))
    
    # Create DP table
    dp = [[0] * (W + 1) for _ in range(n + 1)]
    
    # Fill the DP table
    for i in range(1, n + 1):
        weight, value = items[i - 1]
        for w in range(W + 1):
            # Don't take the item
            dp[i][w] = dp[i - 1][w]
            
            # Take the item if possible
            if weight <= w:
                dp[i][w] = max(dp[i][w], dp[i - 1][w - weight] + value)
    
    print(dp[n][W])

knapsack()

### মূল ধারণা

- প্রতিটি item-এর জন্য ২টা অপশন আছে: নিবো বা নিবো না।
- dp[i][w] মানে: প্রথম iটি item ব্যবহার করে ওজন সীমা w এর মধ্যে সর্বোচ্চ পাওয়া যায় এমন value।
- transition:
    - না নিলে: dp[i][w] = dp[i-1][w]
    - নিলে (যদি weight ≤ w): dp[i][w] = max(dp[i-1][w], dp[i-1][w-weight] + value)
- base case: i = 0 বা w = 0 হলে value = 0 (এটি DP table-এ আগে থেকেই initialize করা আছে)।


আমি একটা 2D DP table বানাইছি যেখানে dp[i][w] মানে হইলো "প্রথম i টা items ইউজ করে w weight limit এর মধ্যে maximum কত value পাওয়া যাবে"। প্রত্যেক item এর জন্য চেক করি যে current weight limit এ fit হবে কিনা। যদি হয়, তাইলে 2 টা জিনিস compare করি - হয় ওই item টা নিবো না (তাইলে value same থাকবে যেমন আগের row এ ছিলো) বা নিবো (তাইলে ওইটার value add হবে বাকি weight এর সাথে)। যেটাতে বেশি value আসে সেইটাই নিই।
Base case হইলো যখন 0 টা items আছে বা 0 capacity আছে, তখন তো value 0 ই হবে, এইটা already initialize করা আছে।

### Complexity:
- Time Complexity: দুইটা nested loop আছে - একটা n items এর জন্য আরেকটা W+1 weights এর জন্য। তাই O(n × W)।
- Space Complexity: 2D array use করতেছি (n+1) × (W+1) size এর, তাই space ও O(n × W)। চাইলে O(W) তে optimize করা যায় একটা array দিয়ে কিন্তু এইভাবে clear বুঝা যায়।

### Problem 2: Shortest Path from Single Source (Dijkstra's Algorithm)

In [ ]:
import heapq

def dijkstra():
    n, m = map(int, input().split())
    
    # Build adjacency list
    graph = [[] for _ in range(n)]
    for _ in range(m):
        u, v, w = map(int, input().split())
        graph[u].append((v, w))
    
    s = int(input())
    
    # Initialize distances
    dist = [float('inf')] * n
    dist[s] = 0
    
    # Priority queue: (distance, vertex)
    pq = [(0, s)]
    
    while pq:
        d, u = heapq.heappop(pq)
        
        # Skip if we already found a better path
        if d > dist[u]:
            continue
        
        # Check all neighbors
        for v, w in graph[u]:
            new_dist = dist[u] + w
            if new_dist < dist[v]:
                dist[v] = new_dist
                heapq.heappush(pq, (new_dist, v))
    
    print(' '.join(map(str, dist)))

dijkstra()

Dijkstra's algorithm টা বেশ সোজা — একটি source থেকে বাকি সব vertices এ shortest path বের করতে হয়। মূল ব্যাপার হলো সবসময় সেই vertex আগে visit করা যেটা সবচেয়ে কাছে থাকে (greedy approach)।

আমি priority queue (min-heap) ব্যবহার করছি যাতে সবসময় minimum distance পাওয়া যায়। Source থেকে শুরু করি distance 0 দিয়ে। প্রতিটি vertex visit করার সময় তার সব neighbours চেক করি এবং দেখি current vertex দিয়ে গেলে কি shorter path পাওয়া যায় কিনা। যদি পাওয়া যায় তাহলে distance update করে queue-তে push করে দিই। গুরুত্বপূর্ণ অংশ হলো আমরা vertex skip করি যদি ইতোমধ্যে ভালো path পাওয়া থাকে (এইটা করে থাকি: `if d > dist[u]`), নাহলে unnecessary কাজ হবে।

### Time Complexity
O((V + E) log V)  
প্রতিটি edge প্রক্রিয়াকরণ করা হয় এবং heap অপারেশনগুলোতে log V সময় লাগে—সর্বোচ্চ E বার heap push হতে পারে।

### Space Complexity
O(V + E)  
Graph adjacency list এ O(V + E), distance array এ O(V), আর priority queue worst-case এ O(E) লাগতে পারে।

### Problem 3: Minimum Spanning Tree (Prim's Algorithm)

In [ ]:
import heapq

def prims():
    n, m = map(int, input().split())
    
    # Build adjacency list
    graph = [[] for _ in range(n)]
    for _ in range(m):
        u, v, w = map(int, input().split())
        graph[u].append((v, w))
        graph[v].append((u, w))  # Undirected graph
    
    # Start from vertex 0
    visited = [False] * n
    min_heap = [(0, 0)]  # (weight, vertex)
    total_weight = 0
    
    while min_heap:
        weight, u = heapq.heappop(min_heap)
        
        # Skip if already visited
        if visited[u]:
            continue
        
        # Mark as visited and add weight to MST
        visited[u] = True
        total_weight += weight
        
        # Add all edges from this vertex to heap
        for v, w in graph[u]:
            if not visited[v]:
                heapq.heappush(min_heap, (w, v))
    
    print(total_weight)

prims()

Prim's algorithm দিয়ে MST বানানো মানে হইলো এমন একটা tree তৈরি করা যেইটা সব vertices connect করবে আর total edge weight minimum হবে। শুরু করি যেকোনো একটা vertex থেকে আর তারপর সবসময় cheapest edge add করতে থাকি যেইটা নতুন একটা vertex connect করে।

Min heap ইউজ করছি যাতে সবসময় minimum weight এর edge পাই। Vertex 0 থেকে শুরু করি, mark করি visited হিসাবে। তারপর যেসব vertices already visited সেইগুলা থেকে যেসব edges বের হইতেছে সেইগুলার মধ্যে সবচেয়ে ছোট যেইটা unvisited vertex এ যায় সেইটা নিই। এইভাবে continue করি যতক্ষণ না সব vertices tree তে add হয়।
Important জিনিস হইলো শুধু unvisited vertices এর জন্য edges add করি, আর already visited vertices skip করি। এইভাবে cycle avoid হয় আর tree বানানো যায়।

#### Time Complexity: O(E log V)
Dijkstra's এর মতোই। Maximum O(E) টা edges heap এ push করি, প্রতিটা push/pop এ O(log E) সময় লাগে যেইটা basically O(log V)। তাই O(E log V)।
#### Space Complexity: O(V + E)
Adjacency list এ O(V + E) লাগে, visited array তে O(V), আর heap এ O(E) edges থাকতে পারে। Overall O(V + E)।

### Problem 4: Kruskal's Algorithm with Disjoint Set Union

In [ ]:
class DSU:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        # Path compression
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        # Union by rank
        px, py = self.find(x), self.find(y)
        
        if px == py:
            return False  # Already in same set
        
        if self.rank[px] < self.rank[py]:
            self.parent[px] = py
        elif self.rank[px] > self.rank[py]:
            self.parent[py] = px
        else:
            self.parent[py] = px
            self.rank[px] += 1
        
        return True

def kruskal():
    n, m = map(int, input().split())
    
    edges = []
    for _ in range(m):
        u, v, w = map(int, input().split())
        edges.append((w, u, v))
    
    # Sort edges by weight
    edges.sort()
    
    dsu = DSU(n)
    total_weight = 0
    edges_used = 0
    
    for w, u, v in edges:
        # Try to add this edge
        if dsu.union(u, v):
            total_weight += w
            edges_used += 1
            
            # MST has n-1 edges
            if edges_used == n - 1:
                break
    
    print(total_weight)

kruskal()

Kruskal's টা Prim's থেকে একটু আলাদা - এইখানে একটা tree grow করার বদলে আমরা সব edges weight অনুযায়ী sort করি আর cheapest edges add করতে থাকি যেইগুলা cycle তৈরি করে না। DSU (Disjoint Set Union) ইউজ করি quickly check করার জন্য যে edge add করলে cycle হবে কিনা।


DSU track রাখে কোন vertices same connected component এ আছে। যদি দুইটা vertices already connected থাকে (same parent), তাইলে তাদের মধ্যে edge add করলে cycle হবে, তাই skip করি। নাহলে edge add করি আর দুইটা sets union করি।
Path compression find() কে fast করে - সব nodes সরাসরি root এর সাথে connect করে tree flatten করে। Union by rank trees balanced রাখে - ছোট tree সবসময় বড় tree এর নিচে attach করে।

#### Time Complexity: O(E log E)
Edges sort করতে O(E log E) লাগে। তারপর edges loop করি আর union-find operations করি। Path compression আর union by rank দিয়ে প্রতিটা operation almost O(1) (technically O(α(n)) যেখানে α হইলো inverse Ackermann, যেইটা practically constant)। তাই overall O(E log E) - sorting dominant।
#### Space Complexity: O(V + E)
সব edges store করতে O(E) লাগে, আর DSU arrays এ O(V)। তাই O(V + E)।

### Problem 5: Grid Pathfinding with Obstacles (DP)

In [ ]:
def grid_paths():
    m, n = map(int, input().split())
    grid = []
    for _ in range(m):
        row = list(map(int, input().split()))
        grid.append(row)
    
    # If start or end is blocked, no paths
    if grid[0][0] == 1 or grid[m-1][n-1] == 1:
        print(0)
        return
    
    # Create DP table
    dp = [[0] * n for _ in range(m)]
    dp[0][0] = 1
    
    # Fill first row
    for j in range(1, n):
        if grid[0][j] == 0:
            dp[0][j] = dp[0][j-1]
    
    # Fill first column
    for i in range(1, m):
        if grid[i][0] == 0:
            dp[i][0] = dp[i-1][0]
    
    # Fill rest of the table
    for i in range(1, m):
        for j in range(1, n):
            if grid[i][j] == 0:
                dp[i][j] = dp[i-1][j] + dp[i][j-1]
    
    print(dp[m-1][n-1])

grid_paths()

এইটা একটা DP problem যেখানে top-left থেকে bottom-right যাওয়ার paths count করতে হয়। শুধু right আর down move করা যায়, আর blocked cells (1) দিয়ে যাওয়া যাবে না।

মূল concept হইলো: cell (i,j) তে পৌঁছানোর জন্য আমরা শুধু উপরের cell (i-1,j) অথবা বামের cell (i,j-1) থেকে আসতে পারি। তাই (i,j) তে পৌঁছানোর ways = (i-1,j) তে পৌঁছানোর ways + (i,j-1) তে পৌঁছানোর ways। কিন্তু শুধু যদি current cell blocked না হয়!

Base case: starting cell (0,0) তে পৌঁছানোর 1 টা way আছে। First row আর column এ শুধু এক direction থেকে আসা যায়, তাই যদি কোনো cell blocked থাকে তাইলে তার পরের সব cells এর 0 ways।
আমরা DP table row by row fill করি, আর answer হইলো dp[m-1][n-1]।

#### Time Complexity: O(m × n)
প্রতিটা cell একবার visit করি আর constant work করি। তাই O(m × n)।
#### Space Complexity: O(m × n)
2D DP table use করতেছি m × n size এর। চাইলে O(n) তে optimize করা যায় শুধু একটা row use করে, কিন্তু এইভাবে clear।